In [209]:
import pandas as pd
import csv
import keras.backend as K
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.constraints import Constraint
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
# from keras.layers import Input
# from keras.layers import LSTM
# from keras.layers import Lambda
from keras import Model
from keras import layers
from keras import Input

class mobility_model(Model):
    def __init__(self, lookback_days=60):
        super(mobility_model, self).__init__()
        self.dense1= Dense(units=256, name='dense1')
#         self.BN1= BatchNormalization()
        self.dropout1= Dropout(0.5)
        self.dense2= Dense(units=64, name='dense2')
        self.dropout2=Dropout(0.5)
        self.dense3= Dense(units=32, name='dense3')
        self.mobility_output = Dense(units=5,
                                   name='final_output')
    def call(self,input_, training=False):
        x=self.dense1(input_)
#         x=self.BN1(x)
        x=self.dropout1(x)
        x=self.dense2(x)
        x=self.dropout2(x)
        x=self.dense3(x)
        x=self.mobility_output(x) 
        return x

In [205]:
import pandas as pd
import csv
import os 
import keras.backend as K
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers
# from ML_mobility import mobility_model
import keras


def train_mobility(path_to_data,lookback_days=60):
    train_df = pd.read_csv(path_to_data)
    train_df=train_df.to_numpy()
    total_x=train_df[:, 2:-5]
    total_y=train_df[:,-5:]
    model_dir="models"
    print(total_y.shape)
    
    total_x = tf.convert_to_tensor(np.asarray(total_x, dtype=np.float64))
    total_y = tf.convert_to_tensor(np.asarray(total_y, dtype=np.float64))
    callbacks_list = [tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, "weights" + "_epoch_{epoch}"),
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=True,
        verbose=1)]

    model=mobility_model(lookback_days)
    opt = keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='mae', optimizer=opt, metrics=["mean_squared_error"])
    
    
    model.fit(total_x,total_y, batch_size=32,validation_split=0.2, epochs = 800,callbacks=callbacks_list, verbose=1)
    print(model.summary())

def predict_mobility(npi_test,lookback_days=60):
    model_dir="models"
    npi_test=npi_test[np.newaxis,:]
    npi_test = tf.convert_to_tensor(np.asarray(npi_test, dtype=np.float64))


    
    model=mobility_model(lookback_days)
    
    latest_checkpoint = tf.train.latest_checkpoint(model_dir)
    
    model.load_weights(latest_checkpoint)
    
    mobility= model.predict(npi_test)
    return mobility

        
    

In [206]:
data_path= "data/train_final_npi.csv"
# data_path= "data/final.csv"
train_mobility(data_path,60)

(2520, 5)
Epoch 1/800
63/63 [==============================] - 0s 3ms/step - loss: 22.7650 - mean_squared_error: 904.6395 - val_loss: 14.6487 - val_mean_squared_error: 388.3080

Epoch 00001: val_loss improved from inf to 14.64872, saving model to models/weights_epoch_1
Epoch 2/800
63/63 [==============================] - 0s 1ms/step - loss: 15.8974 - mean_squared_error: 444.5663 - val_loss: 14.2454 - val_mean_squared_error: 369.8236

Epoch 00002: val_loss improved from 14.64872 to 14.24544, saving model to models/weights_epoch_2
Epoch 3/800
63/63 [==============================] - 0s 1ms/step - loss: 15.4033 - mean_squared_error: 433.8205 - val_loss: 13.8499 - val_mean_squared_error: 353.9228

Epoch 00003: val_loss improved from 14.24544 to 13.84993, saving model to models/weights_epoch_3
Epoch 4/800
63/63 [==============================] - 0s 1ms/step - loss: 14.6739 - mean_squared_error: 403.4332 - val_loss: 13.9245 - val_mean_squared_error: 352.0388

Epoch 00004: val_loss did not im

63/63 [==============================] - 0s 1ms/step - loss: 12.7974 - mean_squared_error: 329.3690 - val_loss: 12.8501 - val_mean_squared_error: 296.3926

Epoch 00013: val_loss did not improve from 12.42799
Epoch 14/800
63/63 [==============================] - 0s 1ms/step - loss: 12.7855 - mean_squared_error: 331.9849 - val_loss: 11.7472 - val_mean_squared_error: 279.4716

Epoch 00014: val_loss improved from 12.42799 to 11.74722, saving model to models/weights_epoch_14
Epoch 15/800
63/63 [==============================] - 0s 1ms/step - loss: 12.8085 - mean_squared_error: 338.2406 - val_loss: 12.6948 - val_mean_squared_error: 292.5543

Epoch 00015: val_loss did not improve from 11.74722
Epoch 16/800
63/63 [==============================] - 0s 1ms/step - loss: 12.4295 - mean_squared_error: 319.3589 - val_loss: 12.5694 - val_mean_squared_error: 289.9832

Epoch 00016: val_loss did not improve from 11.74722
Epoch 17/800
63/63 [==============================] - 0s 1ms/step - loss: 12.1209 -


Epoch 00049: val_loss did not improve from 11.56495
Epoch 50/800
63/63 [==============================] - 0s 1ms/step - loss: 11.2006 - mean_squared_error: 275.9640 - val_loss: 12.3877 - val_mean_squared_error: 306.0037

Epoch 00050: val_loss did not improve from 11.56495
Epoch 51/800
63/63 [==============================] - 0s 1ms/step - loss: 11.4282 - mean_squared_error: 286.2804 - val_loss: 12.5080 - val_mean_squared_error: 307.0552

Epoch 00051: val_loss did not improve from 11.56495
Epoch 52/800
63/63 [==============================] - 0s 1ms/step - loss: 11.2996 - mean_squared_error: 280.0407 - val_loss: 12.2112 - val_mean_squared_error: 308.6989

Epoch 00052: val_loss did not improve from 11.56495
Epoch 53/800
63/63 [==============================] - 0s 1ms/step - loss: 11.4525 - mean_squared_error: 292.6269 - val_loss: 11.8570 - val_mean_squared_error: 305.9380

Epoch 00053: val_loss did not improve from 11.56495
Epoch 54/800
63/63 [==============================] - 0s 1ms/st

63/63 [==============================] - 0s 1ms/step - loss: 10.7700 - mean_squared_error: 259.7640 - val_loss: 12.9702 - val_mean_squared_error: 318.8957

Epoch 00087: val_loss did not improve from 11.56495
Epoch 88/800
63/63 [==============================] - 0s 1ms/step - loss: 11.0332 - mean_squared_error: 270.0951 - val_loss: 12.2869 - val_mean_squared_error: 313.4001

Epoch 00088: val_loss did not improve from 11.56495
Epoch 89/800
63/63 [==============================] - 0s 1ms/step - loss: 10.9726 - mean_squared_error: 272.7881 - val_loss: 12.6956 - val_mean_squared_error: 315.4601

Epoch 00089: val_loss did not improve from 11.56495
Epoch 90/800
63/63 [==============================] - 0s 1ms/step - loss: 11.1867 - mean_squared_error: 274.7250 - val_loss: 12.8528 - val_mean_squared_error: 314.6150

Epoch 00090: val_loss did not improve from 11.56495
Epoch 91/800
63/63 [==============================] - 0s 1ms/step - loss: 11.1168 - mean_squared_error: 273.7016 - val_loss: 12.9

63/63 [==============================] - 0s 1ms/step - loss: 11.0978 - mean_squared_error: 270.1094 - val_loss: 14.0082 - val_mean_squared_error: 348.9575

Epoch 00124: val_loss did not improve from 11.56495
Epoch 125/800
63/63 [==============================] - 0s 2ms/step - loss: 10.9599 - mean_squared_error: 266.0439 - val_loss: 12.4595 - val_mean_squared_error: 313.3604

Epoch 00125: val_loss did not improve from 11.56495
Epoch 126/800
63/63 [==============================] - 0s 1ms/step - loss: 11.1988 - mean_squared_error: 275.7992 - val_loss: 13.6631 - val_mean_squared_error: 339.0472

Epoch 00126: val_loss did not improve from 11.56495
Epoch 127/800
63/63 [==============================] - 0s 1ms/step - loss: 10.9916 - mean_squared_error: 262.0562 - val_loss: 12.8307 - val_mean_squared_error: 318.9987

Epoch 00127: val_loss did not improve from 11.56495
Epoch 128/800
63/63 [==============================] - 0s 1ms/step - loss: 11.0015 - mean_squared_error: 268.1229 - val_loss: 

...

63/63 [==============================] - 0s 2ms/step - loss: 9.8538 - mean_squared_error: 224.1798 - val_loss: 13.1483 - val_mean_squared_error: 360.2590

Epoch 00642: val_loss did not improve from 11.56495
Epoch 643/800
63/63 [==============================] - 0s 1ms/step - loss: 9.6452 - mean_squared_error: 219.9938 - val_loss: 12.6995 - val_mean_squared_error: 354.2729

Epoch 00643: val_loss did not improve from 11.56495
Epoch 644/800
63/63 [==============================] - 0s 1ms/step - loss: 9.9398 - mean_squared_error: 230.2849 - val_loss: 13.0846 - val_mean_squared_error: 355.7067

Epoch 00644: val_loss did not improve from 11.56495
Epoch 645/800
63/63 [==============================] - 0s 2ms/step - loss: 9.6371 - mean_squared_error: 215.8795 - val_loss: 13.1685 - val_mean_squared_error: 359.6996

Epoch 00645: val_loss did not improve from 11.56495
Epoch 646/800
63/63 [==============================] - 0s 1ms/step - loss: 10.0551 - mean_squared_error: 233.0832 - val_loss: 12.7

63/63 [==============================] - 0s 2ms/step - loss: 9.7796 - mean_squared_error: 219.9353 - val_loss: 13.1657 - val_mean_squared_error: 361.8177

Epoch 00679: val_loss did not improve from 11.56495
Epoch 680/800
63/63 [==============================] - 0s 2ms/step - loss: 9.8146 - mean_squared_error: 220.4335 - val_loss: 12.8995 - val_mean_squared_error: 357.7619

Epoch 00680: val_loss did not improve from 11.56495
Epoch 681/800
63/63 [==============================] - 0s 1ms/step - loss: 10.0485 - mean_squared_error: 230.4660 - val_loss: 13.7187 - val_mean_squared_error: 378.1514

Epoch 00681: val_loss did not improve from 11.56495
Epoch 682/800
63/63 [==============================] - 0s 1ms/step - loss: 10.0289 - mean_squared_error: 230.3525 - val_loss: 13.6319 - val_mean_squared_error: 375.7585

Epoch 00682: val_loss did not improve from 11.56495
Epoch 683/800
63/63 [==============================] - 0s 2ms/step - loss: 9.8366 - mean_squared_error: 228.7815 - val_loss: 12.

63/63 [==============================] - 0s 2ms/step - loss: 9.4934 - mean_squared_error: 215.2691 - val_loss: 12.8814 - val_mean_squared_error: 359.7521

Epoch 00716: val_loss did not improve from 11.56495
Epoch 717/800
63/63 [==============================] - 0s 1ms/step - loss: 9.9811 - mean_squared_error: 233.1719 - val_loss: 13.2410 - val_mean_squared_error: 364.5113

Epoch 00717: val_loss did not improve from 11.56495
Epoch 718/800
63/63 [==============================] - 0s 1ms/step - loss: 9.6780 - mean_squared_error: 214.7890 - val_loss: 13.2593 - val_mean_squared_error: 364.4169

Epoch 00718: val_loss did not improve from 11.56495
Epoch 719/800
63/63 [==============================] - 0s 1ms/step - loss: 9.7822 - mean_squared_error: 224.8677 - val_loss: 12.8329 - val_mean_squared_error: 354.0204

Epoch 00719: val_loss did not improve from 11.56495
Epoch 720/800
63/63 [==============================] - 0s 1ms/step - loss: 9.7318 - mean_squared_error: 222.7348 - val_loss: 12.86

63/63 [==============================] - 0s 2ms/step - loss: 9.6488 - mean_squared_error: 219.5079 - val_loss: 13.1381 - val_mean_squared_error: 363.3056

Epoch 00753: val_loss did not improve from 11.56495
Epoch 754/800
63/63 [==============================] - 0s 1ms/step - loss: 9.8118 - mean_squared_error: 226.0830 - val_loss: 12.8825 - val_mean_squared_error: 363.5001

Epoch 00754: val_loss did not improve from 11.56495
Epoch 755/800
63/63 [==============================] - 0s 1ms/step - loss: 9.5887 - mean_squared_error: 220.0216 - val_loss: 13.4975 - val_mean_squared_error: 373.3024

Epoch 00755: val_loss did not improve from 11.56495
Epoch 756/800
63/63 [==============================] - 0s 1ms/step - loss: 9.8055 - mean_squared_error: 226.8202 - val_loss: 12.8482 - val_mean_squared_error: 357.7815

Epoch 00756: val_loss did not improve from 11.56495
Epoch 757/800
63/63 [==============================] - 0s 2ms/step - loss: 9.7084 - mean_squared_error: 223.6002 - val_loss: 13.30

63/63 [==============================] - 0s 2ms/step - loss: 9.7048 - mean_squared_error: 222.5698 - val_loss: 13.0856 - val_mean_squared_error: 362.3123

Epoch 00790: val_loss did not improve from 11.56495
Epoch 791/800
63/63 [==============================] - 0s 2ms/step - loss: 9.5680 - mean_squared_error: 218.1272 - val_loss: 12.8658 - val_mean_squared_error: 359.9019

Epoch 00791: val_loss did not improve from 11.56495
Epoch 792/800
63/63 [==============================] - 0s 1ms/step - loss: 9.7596 - mean_squared_error: 223.2017 - val_loss: 13.2175 - val_mean_squared_error: 367.1565

Epoch 00792: val_loss did not improve from 11.56495
Epoch 793/800
63/63 [==============================] - 0s 1ms/step - loss: 9.6111 - mean_squared_error: 220.6600 - val_loss: 13.4517 - val_mean_squared_error: 373.4236

Epoch 00793: val_loss did not improve from 11.56495
Epoch 794/800
63/63 [==============================] - 0s 2ms/step - loss: 10.0015 - mean_squared_error: 234.2495 - val_loss: 13.5

In [208]:
train_df = pd.read_csv(data_path)
train_df=train_df.to_numpy()
test_x=train_df[190, 2:-5]
print(test_x.shape)
test_y=train_df[190,-5:]
op= predict_mobility(test_x)
print(op)
print(test_y)

(720,)
[[-18.20745  -68.423874 -54.97326   16.800518 -27.447529]]
[-17.857 -68.429 -46.143 15.0 -22.570999999999998]


In [198]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

AttributeError: module 'keras.backend' has no attribute 'tensorflow_backend'